### Importar libs e criar dic de amenities

In [5]:
import requests
import pandas as pd
from time import sleep
from tqdm import tqdm
from datetime import datetime
import config

In [4]:
_AMENITIES = {
        "Kitchen"                       : [2, 8],
        "Wifi"                          : 4,
        "Air conditioning"              : 5,
        "Pool"                          : 7,
        "Free parking on premises"      : 9,
        "Smoking allowed"               : 11,
        "Pets allowed"                  : 12,
        "Gym"                           : 15,
        "Breakfast"                     : 16,
        "Elevator"                      : 21,
        "Hot tub"                       : 25,
        "Indoor fireplace"              : 27,
        "Heating"                       : 30,
        "Washer"                        : 33,
        "Dryer"                         : 34,
        "Smoke alarm"                   : 35,
        "Carbon monoxide alarm"         : 36,
        "Shampoo"                       : 41,
        "Hangers"                       : 44,
        "Hair dryer"                    : 45,
        "Iron"                          : 46,
        "Laptop-friendly workspace"     : 47,
        "Self check-in"                 : 51,
        "TV"                            : 58,
        "High chair"                    : 64,
        "Private bathroom"              : 78,
        "Wide hallways"                 : 109,
        "No stairs or steps to enter"   : [110, 115, 120, 127],
        "Wide entrance for guests"      : 111,
        "Step-free path to entrance"    : 112,
        "Well-lit path to entrance"     : 113,
        "Disabled parking spot"         : 114,
        "Wide entrance"                 : 116,
        "Extra space around bed"        : 117,
        "Accessible-height bed"         : 118,
        "Wide doorway to guest bathroom": 121,
        "Bathtub with bath chair"       : 123,
        "Accessible-height toilet"      : 125,
        "Wide entryway"                 : 128,
        "Handheld shower head"          : 136,
        "Crib"                          : 286,
        "Electric profiling bed"        : 288,
        "Mobile hoist"                  : 289,
        "Pool with pool hoist"          : 290,
        "Ceiling hoist"                 : 291,
        "Fixed grab bars for shower"    : 294,
        "Fixed grab bars for toilet"    : 295,
        "Step-free shower"              : 296,
        "Shower chair"                  : 297,
        "Piano"                         : 347,
        "Extra space around toilet"     : 608,
        "Extra space around shower"     : 609
    }

_AMENITIES = {k.lower() : v for k, v in _AMENITIES.items()}

### Tentativa com pets

In [250]:
# headers = {
#     "X-RapidAPI-Key" : config.API_KEY,
#     "X-RapidAPI-Host": config.API_HOST
#     }
# url = "https://airbnb13.p.rapidapi.com/search-location" # url base da API
# querystring = {"location":"Toronto", "checkin":"2023-05-02", "checkout":"2023-05-20", "adults":"2", "pets":"1", "page":"1", "currency":"BRL"}
# r = requests.get(url, headers=headers, params=querystring).json()

# df_pets = pd.DataFrame(r['results'])

In [ ]:
# for link in df_pets.deeplink:
#     print(link)

In [ ]:
# filter(df_pets, amenities='pets allowed')

### Função de Busca com Parâmetros inseridos pelo usuário

In [20]:
def search(city, checkin, checkout, adults=1, childrens=0, infants=0): # 3 parâmentros obrigatórios e 3 opcionais

    headers = {
    "X-RapidAPI-Key" : config.API_KEY, # token e host fornecidos pelo API na hora do cadastro
    "X-RapidAPI-Host": config.API_HOST
    }

    url = "https://airbnb13.p.rapidapi.com/search-location" # url base da API

    # lista de parâmentros para a pesquisa.    
    querystring = {"location" : city,
                   "checkin" : checkin,
                   "checkout": checkout,
                   "adults"  : adults,
                   "children": childrens,
                   "infants" : infants,
                   "page"    : "1",
                   "currency": "BRL"}

    # Laço para aquisição dos resultados (maxímo de 8 paginas, com 40 resultados por página. limite da API)        
    results = []
    for page in tqdm(range(1,9)):
        querystring["page"] = str(page)
        response = requests.get(url, headers=headers, params=querystring).json()
        if response.get("error") == True or response.get("message") != None:
            print(response.get("message"))
        else:
            results.extend(response["results"])
        sleep(12.5) # A API só deixa fazer 5 requests por minutos.

    # Criação de um DF com o resultado total encontrado
    df = pd.DataFrame(data=results)

    # Algumas informações mostradas ao usuário como uma base
    print(f"{len(results)} results were found.")
    sum = 0
    for ids in results:
        sum += ids["price"]["total"]
    print(f"The avarege price is {querystring['currency']}:{sum/(int(len(results)))}")

    return df


### Inserção das variáveis para a pesquisa no site

#### Testa data CHECKIN

In [18]:
def checkin():
    atual_date = datetime.today()
    var = input("Checkin (Format: dd/mm/aaaa) *Required: ")
    date = datetime.strptime(var,"%d/%m/%Y")
    checkin_date = datetime.strftime(date, "%Y-%m-%d")
    if date < atual_date:
        print("Date can not be in the past!")
        checkin_date = checkin()
    return checkin_date

#### Testa data CHECKOUT

In [17]:
def checkout(checkin_date):
    var = input("Checkout (Format: dd/mm/aaaa) *Required: ")
    date = datetime.strptime(var,"%d/%m/%Y")
    checkin_date = datetime.strptime(checkin_date,"%Y-%m-%d")
    checkout_date = datetime.strftime(date,"%Y-%m-%d")
    if date <= checkin_date:
        print("Checkout must be after checkin")
        checkout_date = checkout()
    return checkout_date

### Entrada dos parâmetros de pesquisa

In [19]:
city = input("Enter the City *Required: ")
checkin = checkin()
checkout = checkout(checkin)
adults = input("Adults (+13y) *Default = 1): ")
childrens = input("Childerns (2-12y) *Default = 0): ")
infants = input("Infants (U-2y) *Default = 0): ")

df = search(city, checkin, checkout, adults, childrens, infants) # Chamada da função com os parâmentros

100%|██████████| 8/8 [01:21<00:00, 10.23s/it]

You have exceeded the rate limit per minute for your plan, BASIC, by the API provider
280 results were found.
The avarege price is BRL:12331.825


### Função Filtro

In [25]:
df.head(5)

,id,url,deeplink,position,name,bathrooms,bedrooms,beds,city,images,...,persons,reviewsCount,type,userId,address,amenityIds,previewAmenities,cancelPolicy,price,rating
0,837253398350534765,https://www.airbnb.com/rooms/837253398350534765,https://www.airbnb.com/rooms/83725339835053476...,1,2*169-B Cozy Suite DowntownToronto,2.0,2,2,Toronto,[https://a0.muscache.com/im/pictures/miso/Host...,...,3,0,Entire home,268420754,"Toronto, ON, Canada","[1, 33, 35, 4, 36, 5, 39, 8, 47]","[Wifi, Kitchen]",CANCEL_STRICT_14_WITH_GRACE_PERIOD,"{'rate': 549, 'currency': 'BRL', 'total': 9867...",NaN
1,845222959531074651,https://www.airbnb.com/rooms/845222959531074651,https://www.airbnb.com/rooms/84522295953107465...,2,2*149-B Private Suite Downtown TO,2.0,2,2,Toronto,[https://a0.muscache.com/im/pictures/miso/Host...,...,3,0,Entire rental unit,312209974,"Toronto, ON, Canada","[1, 35, 4, 36, 5, 39, 8, 47]","[Wifi, Kitchen]",CANCEL_STRICT_14_WITH_GRACE_PERIOD,"{'rate': 608, 'currency': 'BRL', 'total': 1093...",NaN
2,846370000669584955,https://www.airbnb.com/rooms/846370000669584955,https://www.airbnb.com/rooms/84637000066958495...,3,2*53B-1 Cozy Home in Toronto,1.0,2,2,Toronto,[https://a0.muscache.com/im/pictures/miso/Host...,...,4,0,Entire home,311806688,"Toronto, ON, Canada","[1, 35, 4, 36, 5, 39, 8, 40, 45, 30, 47]","[Wifi, Kitchen]",CANCEL_STRICT_14_WITH_GRACE_PERIOD,"{'rate': 549, 'currency': 'BRL', 'total': 9867...",NaN
3,845161151375394620,https://www.airbnb.com/rooms/845161151375394620,https://www.airbnb.com/rooms/84516115137539462...,4,2*169-A Pokemon Downtown Toronto,2.0,3,3,Toronto,[https://a0.muscache.com/im/pictures/miso/Host...,...,5,0,Entire home,268420754,"Toronto, ON, Canada","[1, 33, 35, 4, 36, 5, 39, 8, 47]","[Wifi, Kitchen]",CANCEL_STRICT_14_WITH_GRACE_PERIOD,"{'rate': 746, 'currency': 'BRL', 'total': 1342...",NaN
4,46065895,https://www.airbnb.com/rooms/46065895,https://www.airbnb.com/rooms/46065895?check_in...,5,Modern Renovated Home. Close to subway and Got...,1.0,1,2,Toronto,[https://a0.muscache.com/im/pictures/de4857fd-...,...,4,39,Entire home,103012851,"Toronto, ON, Canada","[1, 4, 5, 8, 77, 79, 16, 657, 85, 86, 89, 90, ...","[Wifi, Kitchen, Self check-in]",CANCEL_STRICT_14_WITH_GRACE_PERIOD,"{'rate': 807, 'currency': 'BRL', 'total': 1451...",4.95


In [264]:
def filter(df, min_bathrooms : float = None, min_bedrooms : int = None, min_beds : int = None, city_only : bool = False, 
           is_superhost : bool = False, rarefind : bool = False, min_person : int = None, min_rvw : int = None, 
           types : str | list = None, max_night_rate : int = None, max_price : int = None, min_rating : float = None, amenities : list = None) -> pd.DataFrame:
    
    # Copiar DF original, para preservá-lo
    df_filter = df.copy()
    currency = df_filter.price[0]['currency']

    # Filtrar parâmetros opcionais
    if min_bathrooms:
        df_filter = df_filter.loc[df_filter.bathrooms >= min_bathrooms].copy()
    if min_bedrooms:
        df_filter = df_filter.loc[df_filter.bedrooms >= min_bedrooms].copy()
    if min_beds:
        df_filter = df_filter.loc[df_filter.beds >= min_beds].copy()
    if city_only:
        df_filter = df_filter.loc[df_filter.city == df_filter.city.value_counts().index[0]].copy()
    if is_superhost:
        df_filter = df_filter.loc[df_filter.isSuperhost == True].copy()
    if rarefind:
        df_filter = df_filter.loc[df_filter.rareFind == True].copy()
    if min_person:
        df_filter = df_filter.loc[df_filter.persons >= min_person].copy()
    if min_rvw:
        df_filter = df_filter.loc[df_filter.reviewsCount >= min_rvw].copy()
    if types:
        if type(types) is str:
            if types.lower() in [t.lower() for t in df_filter.type.unique()]:
                df_filter = df_filter.loc[df_filter.type.apply(str.lower) == types.lower()].copy()
            else:
                return print(f'Type not available, please enter a valid type among the following:\n{df.type.unique()}')
        if type(types) is list:
            types = [i.lower() for i in types]
            check_types = [value in df_filter.type.apply(str.lower).unique() for value in types]
            if any(check_types):
                df_filter = df_filter.loc[df_filter.type.map(lambda value: value.lower() in types)].copy()
            else:
                return print(f'Types not available, please enter a valid type among the following:\n{df.type.unique()}')
            if not all(check_types):
                for value in types:
                    if value not in df_filter.type.apply(str.lower).unique():
                        print(f'{value} is not a valid type and was not used in this filter.')
                print('Please use the following list of types:', df.type.unique(), sep='\n')         
    if max_night_rate:
        df_filter = df_filter.loc[df_filter.price.map(lambda price: price['rate'] <= max_night_rate)].copy()
    if max_price:
        df_filter = df_filter.loc[df_filter.price.map(lambda price: price['total'] <= max_price)].copy()
    if min_rating:
        df_filter = df_filter.loc[df_filter.rating >= min_rating].copy()
    
    # Filtrar amenities
    if amenities:
        wrong_amenity = []
        if type(amenities) is str:
            amenities = [amenities]
        if type(amenities) is list:
            amenities = {a : _AMENITIES.get(a.lower(), False) for a in amenities}
        for amenity, code in amenities.items():
            if type(code) is int:
                df_filter = df_filter.loc[df_filter.amenityIds.map(lambda list: code in list)].copy()
            elif type(code) is list:
                df_filter = df_filter.loc[df_filter.amenityIds.map(lambda list: any([c in list for c in code]))].copy()
            else:
                wrong_amenity.append(amenity)
        if len(wrong_amenity) > 0:
            print(f'The following amenities are not available:\n{wrong_amenity}\nPlease enter a valid amenity among the following:\n{list(_AMENITIES.keys())}')

    # Printar resultados
    total = len(df_filter)
    print(f'\nFound {total} accommodations:')
    if total > 0:
        print('-' * 30)
        print('Minimum night rate:', df_filter.price.map(lambda value: value['rate']).min(), currency)
        print('Maximum night rate:', df_filter.price.map(lambda value: value['rate']).max(), currency)
        l = print('Average night rate:', round(df_filter.price.map(lambda value: value['rate']).mean(), 2), currency)
        print('-' * 30)
        print('Minimum total price:', df_filter.price.map(lambda value: value['total']).min(), currency)
        print('Maximum total price:', df_filter.price.map(lambda value: value['total']).max(), currency)
        print('Average total price:', round(df_filter.price.map(lambda value: value['total']).mean(), 2), currency)

    return df_filter

df_filter = filter(df, min_bathrooms=None, min_bedrooms=None, min_beds=None, city_only=True, 
           is_superhost=False, rarefind=False, min_person=2, min_rvw=None, 
           types=None, max_night_rate=None, max_price=10000, min_rating=None, amenities='pets allowed')


Found 4 accommodations:
------------------------------
Minimum night rate: 484 BRL
Maximum night rate: 555 BRL
Average night rate: 518.5 BRL
------------------------------
Minimum total price: 8700 BRL
Maximum total price: 9989 BRL
Average total price: 9324.75 BRL


In [265]:
df_filter

,id,url,deeplink,position,name,bathrooms,bedrooms,beds,city,images,...,persons,reviewsCount,type,userId,address,amenityIds,previewAmenities,cancelPolicy,price,rating
24,833290035655790218,https://www.airbnb.com/rooms/833290035655790218,https://www.airbnb.com/rooms/83329003565579021...,25,One Bedroom street level unit/separate entrance,1.0,1,1,Toronto,[https://a0.muscache.com/im/pictures/miso/Host...,...,3,0,Entire home,275771629,"Toronto, ON, Canada","[1, 33, 129, 35, 4, 36, 5, 39, 8, 12]","[Wifi, Kitchen]",CANCEL_MODERATE,"{'rate': 547, 'currency': 'BRL', 'total': 9829...",NaN
38,30851612,https://www.airbnb.com/rooms/30851612,https://www.airbnb.com/rooms/30851612?check_in...,39,Guest Suite - Bright and Modern House off 401,1.0,1,1,Toronto,[https://a0.muscache.com/im/pictures/f4596edf-...,...,3,146,Private room in bungalow,230664361,"Toronto, ON, Canada","[1, 2, 4, 5, 8, 9, 137, 394, 522, 12, 77, 85, ...","[Wifi, Free parking, Kitchen, Self check-in]",CANCEL_BETTER_STRICT_WITH_GRACE_PERIOD,"{'rate': 484, 'currency': 'BRL', 'total': 8700...",4.84
97,832860306325352581,https://www.airbnb.com/rooms/832860306325352581,https://www.airbnb.com/rooms/83286030632535258...,98,Private Rooms /4Pl /2 Q Beds#B,1.0,2,2,Toronto,[https://a0.muscache.com/im/pictures/miso/Host...,...,4,2,Private room in home,487846315,"Toronto, ON, Canada","[1, 33, 34, 35, 4, 36, 100, 5, 37, 101, 8, 9, ...","[Wifi, Free parking, Kitchen, Self check-in]",CANCEL_FLEXIBLE,"{'rate': 488, 'currency': 'BRL', 'total': 8781...",NaN
191,42711635,https://www.airbnb.com/rooms/42711635,https://www.airbnb.com/rooms/42711635?check_in...,192,Recently renovated super clean Studio Apartmen...,1.0,0,1,Toronto,[https://a0.muscache.com/im/pictures/cc16bda8-...,...,3,88,Entire guest suite,3229206,"Toronto, ON, Canada","[1, 66, 4, 8, 12, 77, 15, 79, 657, 146, 85, 88...","[Wifi, Kitchen, Self check-in]",CANCEL_BETTER_STRICT_WITH_GRACE_PERIOD,"{'rate': 555, 'currency': 'BRL', 'total': 9989...",4.66


In [266]:
for link in df_filter.deeplink:
    print(link)

https://www.airbnb.com/rooms/833290035655790218?check_in=2023-05-02&check_out=2023-05-20&adults=2&children=0&infants=0
https://www.airbnb.com/rooms/30851612?check_in=2023-05-02&check_out=2023-05-20&adults=2&children=0&infants=0
https://www.airbnb.com/rooms/832860306325352581?check_in=2023-05-02&check_out=2023-05-20&adults=2&children=0&infants=0
https://www.airbnb.com/rooms/42711635?check_in=2023-05-02&check_out=2023-05-20&adults=2&children=0&infants=0
